In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [2]:
%matplotlib inline
sns.set_style(style='whitegrid')

In [3]:
import bin_keys
from binance.client import Client

In [4]:
client = Client(api_key=bin_keys.apiKey, api_secret=bin_keys.secretKey)

In [5]:
candles = client.get_klines(symbol='LTCUSDT', interval=Client.KLINE_INTERVAL_1MINUTE, limit=2000)

In [6]:
columns = ['Open Time', 'Open', 'High', 'Low', 'Close',
            'Volume', 'Close Time', 'Quote asset volume', 
            'n_trades', 'Taker buy base asset volume', 
            'Taker buy quote asset volume', 'Ignore']

In [7]:
ltc_usdt_df = pd.DataFrame(candles, columns=columns)

In [8]:
ltc_usdt_df.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote asset volume,n_trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,1582929540000,60.26000000,60.34000000,60.25000000,60.29000000,533.74720000,1582929599999,32185.51656320,141,478.45478000,28852.02915970,0
1,1582929600000,60.32000000,60.42000000,60.30000000,60.42000000,511.21730000,1582929659999,30867.02726740,165,385.90142000,23301.12160630,0
2,1582929660000,60.40000000,60.43000000,60.32000000,60.35000000,284.04923000,1582929719999,17146.87510010,74,156.74457000,9463.93841160,0
3,1582929720000,60.35000000,60.39000000,60.29000000,60.39000000,163.07259000,1582929779999,9836.22396530,91,71.59456000,4319.97480640,0
4,1582929780000,60.39000000,60.48000000,60.36000000,60.45000000,434.31424000,1582929839999,26255.40850970,159,377.24694000,22807.87605810,0


In [9]:
ltc_usdt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Open Time                     1000 non-null   int64 
 1   Open                          1000 non-null   object
 2   High                          1000 non-null   object
 3   Low                           1000 non-null   object
 4   Close                         1000 non-null   object
 5   Volume                        1000 non-null   object
 6   Close Time                    1000 non-null   int64 
 7   Quote asset volume            1000 non-null   object
 8   n_trades                      1000 non-null   int64 
 9   Taker buy base asset volume   1000 non-null   object
 10  Taker buy quote asset volume  1000 non-null   object
 11  Ignore                        1000 non-null   object
dtypes: int64(3), object(9)
memory usage: 93.9+ KB


In [10]:
ltc_usdt_df[['Open','Close', 'High','Low', 'Volume', 'Quote asset volume', 
            'Taker buy base asset volume', 'Taker buy quote asset volume'
           ]] = ltc_usdt_df[['Open','Close', 'High','Low', 'Volume', 'Quote asset volume', 
            'Taker buy base asset volume', 'Taker buy quote asset volume']].astype(float)

In [11]:
ltc_usdt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Open Time                     1000 non-null   int64  
 1   Open                          1000 non-null   float64
 2   High                          1000 non-null   float64
 3   Low                           1000 non-null   float64
 4   Close                         1000 non-null   float64
 5   Volume                        1000 non-null   float64
 6   Close Time                    1000 non-null   int64  
 7   Quote asset volume            1000 non-null   float64
 8   n_trades                      1000 non-null   int64  
 9   Taker buy base asset volume   1000 non-null   float64
 10  Taker buy quote asset volume  1000 non-null   float64
 11  Ignore                        1000 non-null   object 
dtypes: float64(8), int64(3), object(1)
memory usage: 93.9+ KB


In [12]:
ltc_usdt_df['Close Time'] = ltc_usdt_df['Close Time'].apply(
    lambda x: datetime.fromtimestamp(int(x/1000)))

In [13]:
ltc_usdt_df['Open Time'] = ltc_usdt_df['Open Time'].apply(
    lambda x: datetime.fromtimestamp(int(x/1000)))

In [14]:
ltc_usdt_df.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote asset volume,n_trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,2020-02-28 23:39:00,60.26,60.34,60.25,60.29,533.74720,2020-02-28 23:39:59,32185.516563,141,478.45478,28852.029160,0
1,2020-02-28 23:40:00,60.32,60.42,60.30,60.42,511.21730,2020-02-28 23:40:59,30867.027267,165,385.90142,23301.121606,0
2,2020-02-28 23:41:00,60.40,60.43,60.32,60.35,284.04923,2020-02-28 23:41:59,17146.875100,74,156.74457,9463.938412,0
3,2020-02-28 23:42:00,60.35,60.39,60.29,60.39,163.07259,2020-02-28 23:42:59,9836.223965,91,71.59456,4319.974806,0
4,2020-02-28 23:43:00,60.39,60.48,60.36,60.45,434.31424,2020-02-28 23:43:59,26255.408510,159,377.24694,22807.876058,0


In [15]:
ltc_usdt_df['Open Time'].unique().max()

numpy.datetime64('2020-02-29T16:18:00.000000000')

In [16]:
ltc_usdt_df['Open Time'].unique().min()

numpy.datetime64('2020-02-28T23:39:00.000000000')